# Advanced Classification - Day 6 - Exercises

## Exercise 1

#### Question 1 
##### Load libraries that are used in this module.

#### Question 2 
##### Set `main_dir` to the location of your `af-werx` folder.
##### Make `data_dir` from the `main_dir` and concatenate remainder of the path to data directory.

#### Question 3
##### Set the working directory to data_dir.
##### Check if the working directory is updated to `data_dir'.

#### Question 4
##### Read in 'lastfm_ratings.csv' dataset to a dataframe named 'fm_ratings' and 'lastfm_artists.csv' as 'fm_artists'.
##### Print the info and head of the dataset.

#### Question 5
##### Print a word cloud to find which words are repeated in the artist name using fm_artists dataframe.

#### Question 6
##### Find the distribution of rating by plotting a count plot in fm_ratings dataframe.

#### Question 7
##### Fetch all the genres in the dataset and print top 5 using the function we used in the module.

In [21]:
# Function that counts the number of times each of the genre keywords appear.
def count_word(dataset, ref_col, census):
    keyword_count = dict()
    for s in census: 
        keyword_count[s] = 0
    for census_keywords in dataset[ref_col].str.split('|'):        
        if type(census_keywords) == float and pd.isnull(census_keywords): 
            continue        
        for s in [s for s in census_keywords if s in census]: 
            if pd.notnull(s): 
                keyword_count[s] += 1

    # Convert the dictionary in a list to sort the keywords by frequency.
    keyword_occurrences = []
    for k,v in keyword_count.items():
        keyword_occurrences.append([k,v])
    keyword_occurrences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurrences, keyword_count

#### Question 8
##### Find the word cloud for the genres to see which genres are listed more frequently.

### Exercise 2

#### Question 1
##### We will build a content based recommender system using `fm_artists` dataframe.
##### Convert the genre to string array.
##### Calculate the TF-IDF matrix.

#### Question 2
##### Build the cosine similarity of the artists, call it 'cosine_sim', and print the top entries in that matrix.
##### Also, print the shape of the cosine similarity matrix.
##### Build the list of artists' titles, and label it 'indices'.

#### Question 3
##### Use the function to return top similar artists based on the cosine similarity value for any given artist name.
##### Find recommendations for a person who likes 'Rihanna' in their playlist.
##### Also, find recommendations for someone who likes 'Madonna'.

In [31]:
# Function that get artists recommendations based on the cosine similarity score of artists.
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

### Exercise 3

#### Question 1
##### Transform the fm_ratings with userID as the row and artist_name as the column and the rating as the value. Set it as "userRating".
##### Find the correlation matrix for the artist_name. Do not use min_periods as we did in module, since our dataset is small here.
##### It could take a lot of time to form the correlation matrix, so you can also load from our data_dir where we already have it calculated and saved.
##### Load "corrMatrix_ex.csv" as corrMatrix - also, set the first column `artist_name` as the index for the dataframe.
##### Hint: df = df.set_index('artist_name')

#### Question 2
##### We will find recommendations for userID 25. Assign user_id as 25 and use the same steps we did in the module to find the artist recommendation.
##### First, create a list of all artists with all correlations multiplied by rating.
##### Group by artist_id and sum the ratings to remove the duplicates.

#### Question 3
##### Find the list of artists that the user has already heard and remove them.
##### Give top 10 recommendations.

### Exercise 4

#### Question 1
##### Find the total number of users and artists.
##### Transform our fm_ratings dataset using pivot_table where we have 1 row per user and 1 column per artist.

#### Question 2
##### Convert the pivot table into matrix and find the sparsity percentage.

#### Question 3
##### Fetch the first 50 latent features and return the list of artists the user has already rated.


#### Question 4
##### The following function has been modified from the function we used in the module to recommend artists.
##### Use this function and predict 20 new artists to user with ID 400.

In [47]:
def recommend_songs(predictions, user, artists, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions.
    user_row_number = user - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the artist information.
    user_data = original_ratings[original_ratings.userID == (user)]
    user_full = (user_data.merge(fm_artists, how = 'left', left_on = 'artistID', right_on = 'artistID').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} artists.'.format(user, user_full.shape[0]))
    print('Recommending highest {0} predicted rating artists not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating artists that the user hasn't listened yet.
    recommendations = (fm_artists[~fm_artists['artistID'].isin(user_full['artistID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'artistID',
               right_on = 'artistID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

#### Question 5
##### Load the reader library and load the dataset `fm_ratings` to reader.
##### We will do a 5-fold evaluation.

#### Question 6
##### Build the SVD algorithm and evaluate with our data using RMSE metric.
##### What are the RMSE values?

#### Question 7
##### Using our algorithm, predict what would be the rating given by user 1200 to artist ID 400.